In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset # Import Subset
import torchvision.transforms as T # Use alias T
import timm
from sklearn.metrics import mean_squared_error # Keep for potential checks
from tqdm import tqdm
import gc
import time
import math
import copy # Needed for deepcopying model state
import warnings

# Suppress specific warnings if needed
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

# --- Configuration ---
BASE_DIR = "/kaggle/input/"
# Adjust dataset paths if necessary - Use correct paths provided by user in previous request
TRAIN_CSV_PATH = os.path.join(BASE_DIR, "iiith-campus/labels_train.csv") # Corrected path
VAL_CSV_PATH = os.path.join(BASE_DIR, "iiith-campus/labels_val.csv")
TRAIN_IMG_DIR = os.path.join(BASE_DIR, "iiith-campus/images_train/images_train")
VAL_IMG_DIR = os.path.join(BASE_DIR, "iiith-campus/images_val/images_val")
TEST_IMG_DIR = os.path.join(BASE_DIR, "images-test/images_test") # Common Kaggle test image path structure
# Path to YOUR trained region classifier
REGION_CLASSIFIER_PATH = os.path.join(BASE_DIR, "region_classifier/pytorch/default/1/final_campus_region_model.pth") # Make sure this is correct

# Model save path (Optional - not strictly needed as we keep models in memory)
WORK_DIR = "/kaggle/working/"
os.makedirs(WORK_DIR, exist_ok=True)

REGION_CLASSIFIER_ARCH = 'convnextv2_tiny.fcmae_ft_in1k' # Arch of the region classifier
REGRESSOR_ARCH = 'convnextv2_tiny.fcmae_ft_in1k'       # Base arch for the 15 regressors

IMG_SIZE = 224 # ConvNeXtV2 Tiny uses 224
BATCH_SIZE = 32 # Batch size for training EACH region model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_REGIONS = 15
NUM_WORKERS = 2 # Use 2 workers for Kaggle usually

# --- Region-Specific Regressor Training Hyperparameters ---
N_EPOCHS_PER_REG_MODEL = 30 # Number of epochs to train each region's ConvNeXt regressor
LEARNING_RATE_REGRESSOR = 1e-4 # LR for the region-specific regressors
WEIGHT_DECAY_REGRESSOR = 1e-5
MIN_SAMPLES_PER_REGION = 20 # Minimum samples needed to train a region model

# --- NO IGNORE INDICES FOR VALIDATION IN THIS TASK ---
SEED = 42

# --- Seeding and Print Config ---
print(f"--- Configuration ---")
print(f"Using device: {DEVICE}")
print(f"Region Classifier Arch: {REGION_CLASSIFIER_ARCH}")
print(f"Regressor Base Arch: {REGRESSOR_ARCH}")
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Epochs per Region Model: {N_EPOCHS_PER_REG_MODEL}")
print(f"Regressor LR: {LEARNING_RATE_REGRESSOR}")
print(f"Min Samples per Region: {MIN_SAMPLES_PER_REGION}")
print(f"Train CSV Path: {TRAIN_CSV_PATH}")
print(f"Val CSV Path: {VAL_CSV_PATH}")
print(f"Train Image Dir: {TRAIN_IMG_DIR}")
print(f"Val Image Dir: {VAL_IMG_DIR}")
print(f"Test Image Dir: {TEST_IMG_DIR}")
print(f"---------------------\n")
torch.manual_seed(SEED); np.random.seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed(SEED); torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False

# --- MAAE Calculation & Degree Conversion Functions ---
def calculate_maae(y_true, y_pred):
    if isinstance(y_true, torch.Tensor): y_true = y_true.cpu().numpy()
    if isinstance(y_pred, torch.Tensor): y_pred = y_pred.cpu().numpy()
    y_true = np.array(y_true) % 360; y_pred = np.array(y_pred) % 360
    valid_idx = ~np.isnan(y_pred) & ~np.isnan(y_true) & np.isfinite(y_pred) & np.isfinite(y_true)
    if np.sum(valid_idx) == 0: return np.nan
    y_true, y_pred = y_true[valid_idx], y_pred[valid_idx]
    if len(y_true) == 0: return np.nan
    diff = np.abs(y_true - y_pred); angular_error = np.minimum(diff, 360.0 - diff)
    return np.mean(angular_error)
def predictions_to_degrees(outputs_cos_sin):
    # Ensure input is tensor for torch.atan2
    if isinstance(outputs_cos_sin, np.ndarray): outputs_cos_sin = torch.from_numpy(outputs_cos_sin)
    outputs_cos_sin_cpu = outputs_cos_sin.cpu()
    pred_rad = torch.atan2(outputs_cos_sin_cpu[:, 1], outputs_cos_sin_cpu[:, 0]) # atan2(sin, cos)
    pred_deg = torch.rad2deg(pred_rad)
    return (pred_deg + 360.0) % 360.0

# --- 1. Load Pre-trained Region Classifier (Frozen) ---
print(f"Loading Frozen Region Classifier: {REGION_CLASSIFIER_ARCH}")
try:
    region_classifier_frozen = timm.create_model(REGION_CLASSIFIER_ARCH, pretrained=False, num_classes=NUM_REGIONS)
    if not os.path.exists(REGION_CLASSIFIER_PATH): raise FileNotFoundError(f"Region classifier weights not found: {REGION_CLASSIFIER_PATH}")
    checkpoint = torch.load(REGION_CLASSIFIER_PATH, map_location='cpu'); state_dict = checkpoint.get('state_dict', checkpoint.get('model', checkpoint))
    state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    try: region_classifier_frozen.load_state_dict(state_dict, strict=True)
    except RuntimeError: region_classifier_frozen.load_state_dict(state_dict, strict=False)
    for param in region_classifier_frozen.parameters(): param.requires_grad = False
    region_classifier_frozen.to(DEVICE); region_classifier_frozen.eval()
    print("Region classifier loaded and frozen successfully.")
except Exception as e: print(f"FATAL ERROR loading region classifier: {e}"); raise


# --- 2. Define Transforms ---
print("Defining Transformations...")
try:
    dummy_reg_model = timm.create_model(REGRESSOR_ARCH, pretrained=True)
    data_config = timm.data.resolve_model_data_config(model=dummy_reg_model)
    data_config['input_size'] = (3, IMG_SIZE, IMG_SIZE)
    transform_train = T.Compose([
        T.Resize((IMG_SIZE + 32, IMG_SIZE + 32), interpolation=T.InterpolationMode.BICUBIC),
        T.RandomCrop(IMG_SIZE), T.RandomHorizontalFlip(p=0.5),
        T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
        T.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=5),
        T.ToTensor(), T.Normalize(mean=data_config['mean'], std=data_config['std']),
        T.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3))])
    transform_eval = timm.data.create_transform(**data_config, is_training=False)
    print(f" Transforms created (Input Size: {IMG_SIZE}x{IMG_SIZE}). Includes Training Augmentation.")
    del dummy_reg_model
except Exception as e:
    print(f"Error creating timm transforms: {e}. Using basic fallback.")
    transform_train = T.Compose([ T.Resize((IMG_SIZE, IMG_SIZE)), T.RandomHorizontalFlip(), T.ColorJitter(0.1,0.1,0.1), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    transform_eval = T.Compose([ T.Resize((IMG_SIZE, IMG_SIZE)), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


# --- 3. Define Dataset for Angle Regression ---
class AngleRegressionDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, is_train=False):
        self.img_dir = img_dir; self.transform = transform; self.is_train = is_train
        try: df_raw = pd.read_csv(csv_file)
        except FileNotFoundError: print(f"ERROR: CSV {csv_file} not found."); raise
        df_raw['angle'] = pd.to_numeric(df_raw['angle'], errors='coerce')
        df_raw = df_raw.dropna(subset=['angle', 'filename'])
        if is_train:
            original_len = len(df_raw); self.df = df_raw[(df_raw['angle'] >= 0) & (df_raw['angle'] <= 360)].copy(); filtered_len = len(self.df); print(f"Filtered training data: {original_len} -> {filtered_len} rows.")
        else: self.df = df_raw.copy()
        self.df.reset_index(drop=True, inplace=True); self.df['original_index'] = self.df.index
        if 'Region_ID' not in self.df.columns: print(f"FATAL ERROR: 'Region_ID' column not found in {csv_file}"); raise KeyError
        self.df['region_id_0idx'] = self.df['Region_ID'] - 1
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        if idx >= len(self.df): raise IndexError()
        row = self.df.iloc[idx]; img_name = row['filename']; original_idx = row['original_index']; img_path = os.path.join(self.img_dir, img_name)
        angle_deg = row['angle']; angle_rad = math.radians(angle_deg); sin_angle = math.sin(angle_rad); cos_angle = math.cos(angle_rad)
        target = torch.tensor([cos_angle, sin_angle], dtype=torch.float32)
        true_region_id = row['region_id_0idx']
        try: image = Image.open(img_path).convert('RGB')
        except Exception: return None
        if self.transform:
            try: image = self.transform(image)
            except Exception: return None
        return image, target, angle_deg, true_region_id, original_idx

def angle_reg_collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: return None
    images, targets, angles_deg, true_region_ids, original_indices = zip(*batch)
    return torch.stack(images, 0), torch.stack(targets, 0), \
           torch.tensor(angles_deg, dtype=torch.float32), \
           torch.tensor(true_region_ids, dtype=torch.long), \
           torch.tensor(original_indices, dtype=torch.long)

# --- Create the FULL Training and Validation Datasets ---
print("\nLoading full training dataset...")
full_train_dataset = AngleRegressionDataset(TRAIN_CSV_PATH, TRAIN_IMG_DIR, transform=transform_train, is_train=True)
print("Loading full validation dataset...")
full_val_dataset = AngleRegressionDataset(VAL_CSV_PATH, VAL_IMG_DIR, transform=transform_eval, is_train=False) # Use eval transform
print(f"Full training dataset size: {len(full_train_dataset)}")
print(f"Full validation dataset size: {len(full_val_dataset)}")


# --- 4. Train Region-Specific ConvNeXt Regressors ---
def train_region_convnext_regressors(full_train_ds, full_val_ds, # Pass both datasets
                                     n_regions=NUM_REGIONS, base_model_arch=REGRESSOR_ARCH,
                                     epochs=N_EPOCHS_PER_REG_MODEL, lr=LEARNING_RATE_REGRESSOR, wd=WEIGHT_DECAY_REGRESSOR,
                                     min_samples=MIN_SAMPLES_PER_REGION, device=DEVICE):
    trained_regressors = {}
    print(f"\n--- Training {n_regions} Region-Specific ConvNeXt Regressors ---")
    if not hasattr(full_train_ds, 'df'): raise AttributeError("Training dataset object missing 'df' attribute.")
    if not hasattr(full_val_ds, 'df'): raise AttributeError("Validation dataset object missing 'df' attribute.")
    all_train_true_region_ids = np.array(full_train_ds.df['region_id_0idx'])
    all_val_true_region_ids = np.array(full_val_ds.df['region_id_0idx'])

    for region_id in range(n_regions):
        print(f"\n-- Training Regressor for Region {region_id + 1}/{n_regions} --")
        # --- Training Data Setup ---
        train_region_indices = np.where(all_train_true_region_ids == region_id)[0]
        if len(train_region_indices) < min_samples:
            print(f"Skipping Region {region_id + 1}: Only {len(train_region_indices)} train samples found (min required {min_samples}).")
            trained_regressors[region_id] = None; continue
        print(f" Training with {len(train_region_indices)} samples.")
        train_region_subset = Subset(full_train_ds, train_region_indices)
        train_region_loader = DataLoader(train_region_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=angle_reg_collate_fn, drop_last=True)
        if len(train_region_loader) == 0:
             print(f"Skipping Region {region_id + 1}: Not enough train samples for a full batch after drop_last."); trained_regressors[region_id] = None; continue

        # --- Validation Data Setup for this region ---
        val_region_loader = None; val_region_subset = None
        val_region_indices = np.where(all_val_true_region_ids == region_id)[0]
        if len(val_region_indices) > 0:
             val_region_subset = Subset(full_val_ds, val_region_indices)
             val_region_loader = DataLoader(val_region_subset, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=NUM_WORKERS, collate_fn=angle_reg_collate_fn)
             print(f" Validating with {len(val_region_indices)} samples for this region.")
        else: print(" No validation samples found for this region.")

        # Instantiate Model, Optimizer, Loss
        model = timm.create_model(base_model_arch, pretrained=True)
        if hasattr(model, 'get_classifier'): num_features = model.get_classifier().in_features; model.reset_classifier(2);
        elif hasattr(model, 'head'): num_features = model.head.fc.in_features; model.head.fc = nn.Linear(num_features, 2)
        else: raise AttributeError(f"Cannot find classifier layer in {base_model_arch}")
        model = model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        criterion = nn.MSELoss()

        # *** Initialize tracking for best model state within this region ***
        best_region_val_maae = float('inf')
        best_region_model_state = None
        best_region_epoch = -1

        # Training loop
        for epoch in range(epochs):
            model.train(); epoch_loss = 0.0; epoch_samples = 0
            epoch_train_true_angles = []; epoch_train_pred_angles = []
            pbar_region = tqdm(train_region_loader, desc=f"Reg {region_id+1} Ep {epoch+1}/{epochs} [T]", leave=False)
            for batch_data in pbar_region:
                if batch_data is None: continue
                images, targets_cos_sin, true_angles_deg, _, _ = batch_data
                images, targets_cos_sin = images.to(device), targets_cos_sin.to(device)
                optimizer.zero_grad(); outputs_cos_sin = model(images); loss = criterion(outputs_cos_sin, targets_cos_sin)
                if not torch.isfinite(loss): continue
                loss.backward(); optimizer.step()
                epoch_loss += loss.item() * images.size(0); epoch_samples += images.size(0)
                with torch.no_grad(): pred_angles_deg = predictions_to_degrees(outputs_cos_sin); epoch_train_pred_angles.extend(pred_angles_deg.cpu().tolist()); epoch_train_true_angles.extend(true_angles_deg.cpu().tolist())
                if epoch_samples > 0: pbar_region.set_postfix(loss=f"{epoch_loss / epoch_samples:.4f}")
            pbar_region.close()

            # --- Epoch Validation (for this region) ---
            epoch_val_maae = np.nan
            if val_region_loader is not None and len(val_region_loader) > 0:
                model.eval(); epoch_val_true_angles = []; epoch_val_pred_angles = []
                with torch.no_grad():
                    for batch_data in val_region_loader:
                        if batch_data is None: continue
                        images, _, true_angles_deg, _, _ = batch_data; images = images.to(device)
                        outputs_cos_sin = model(images); pred_angles_deg = predictions_to_degrees(outputs_cos_sin)
                        epoch_val_pred_angles.extend(pred_angles_deg.cpu().tolist()); epoch_val_true_angles.extend(true_angles_deg.cpu().tolist())
                epoch_val_maae = calculate_maae(epoch_val_true_angles, epoch_val_pred_angles)

                # *** Check if this epoch is the best for this region ***
                if np.isfinite(epoch_val_maae) and epoch_val_maae < best_region_val_maae:
                    best_region_val_maae = epoch_val_maae
                    best_region_model_state = copy.deepcopy(model.state_dict())
                    best_region_epoch = epoch + 1
                    # *** ADDED PRINT STATEMENT ***
                    print(f"   ---> New best Val MAAE for region {region_id+1}: {best_region_val_maae:.4f} at epoch {best_region_epoch}")

            # Print Epoch Metrics
            avg_epoch_loss = epoch_loss / epoch_samples if epoch_samples > 0 else float('nan')
            epoch_train_maae = calculate_maae(epoch_train_true_angles, epoch_train_pred_angles)
            print(f"  Reg {region_id+1} - Ep {epoch+1}/{epochs} - Train Loss(MSE): {avg_epoch_loss:.4f} | Train MAAE: {epoch_train_maae:.4f} | Val MAAE: {epoch_val_maae:.4f}")

        # --- Load best state AFTER all epochs for the region are done ---
        final_model = timm.create_model(base_model_arch, pretrained=False, num_classes=2) # Create structure again
        if best_region_model_state is not None:
            print(f"  Loading best model state for region {region_id+1} from epoch {best_region_epoch} (Val MAAE: {best_region_val_maae:.4f})")
            final_model.load_state_dict(best_region_model_state)
        else:
             print(f"  Warning: No best validation state found for region {region_id+1}. Using last epoch state.")
             # Need to load the last state back into final_model if best_state is None
             # Since model holds the last state, we can copy it here, or ideally save last state too
             # Simplified: If no best, the 'model' variable still holds the last state, but we need to put it in final_model
             # Re-create the model and load the *last* state dict if no best was found
             model_last_state = timm.create_model(base_model_arch, pretrained=False, num_classes=2)
             if hasattr(model_last_state, 'get_classifier'): num_features = model_last_state.get_classifier().in_features; model_last_state.reset_classifier(2);
             elif hasattr(model_last_state, 'head'): num_features = model_last_state.head.fc.in_features; model_last_state.head.fc = nn.Linear(num_features, 2)
             else: raise AttributeError(f"Cannot find classifier layer in {base_model_arch}")
             model_last_state.load_state_dict(model.state_dict()) # Load state from end of training
             final_model = model_last_state # Use the last state model

        final_model.to(device).eval() # Move to device and set to eval
        trained_regressors[region_id] = final_model
        print(f"  Finished training & storing best/last regressor for Region {region_id + 1}.")

        # Cleanup
        del model, optimizer, train_region_loader, train_region_subset
        if val_region_loader is not None: del val_region_loader, val_region_subset
        if best_region_model_state is not None: del best_region_model_state
        gc.collect(); torch.cuda.empty_cache()

    print("\n--- Finished Training All Region Regressors ---")
    return trained_regressors

# Train the 15 models
trained_regressors = train_region_convnext_regressors(full_train_dataset, full_val_dataset)


# --- 5. Evaluate Overall Performance on Validation Set ---
def evaluate_overall_performance(val_csv_path, val_img_dir, transform_eval,
                                 frozen_region_classifier, trained_regressors_dict,
                                 device=DEVICE, batch_size=BATCH_SIZE*2):
    print("\n--- Evaluating Overall Performance on Full Validation Set ---")
    eval_dataset = AngleRegressionDataset(val_csv_path, val_img_dir, transform=transform_eval, is_train=False)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS, collate_fn=angle_reg_collate_fn)
    all_val_preds_deg = []; all_val_trues_deg = []; missing_regressor_count = 0
    if len(eval_loader) == 0: print("Validation loader is empty."); return np.nan
    frozen_region_classifier.eval()
    with torch.no_grad():
        for batch_data in tqdm(eval_loader, desc="Overall Validation Eval"):
            if batch_data is None: continue
            images, _, true_angles_deg, _, _ = batch_data; images = images.to(device)
            region_logits = frozen_region_classifier(images); pred_region_ids = torch.argmax(region_logits, dim=1).cpu().numpy()
            for i in range(images.size(0)):
                pred_region_id = pred_region_ids[i]; regressor_model = trained_regressors_dict.get(pred_region_id)
                img_single = images[i:i+1]; true_angle_single = true_angles_deg[i].item(); pred_angle = 0.0 # Default
                if regressor_model is not None:
                    try:
                        regressor_model.eval(); outputs_cos_sin = regressor_model(img_single); pred_angle_tensor = predictions_to_degrees(outputs_cos_sin)
                        pred_angle = pred_angle_tensor.item();
                        if not np.isfinite(pred_angle): pred_angle = 0.0
                    except Exception as e: pred_angle = 0.0 # Fallback on error
                else: missing_regressor_count += 1; pred_angle = 0.0 # Fallback if model missing
                all_val_preds_deg.append(pred_angle); all_val_trues_deg.append(true_angle_single)
    final_val_maae = calculate_maae(all_val_trues_deg, all_val_preds_deg)
    print(f"\nFinal Overall Validation MAAE (All Samples): {final_val_maae:.4f}")
    if missing_regressor_count > 0: print(f"  (Regressor model was missing for {missing_regressor_count} predictions)")
    return final_val_maae

final_maae = evaluate_overall_performance(VAL_CSV_PATH, VAL_IMG_DIR, transform_eval, region_classifier_frozen, trained_regressors)


# ==============================================================================
# >> Submission CSV Generation <<
# ==============================================================================
print("\n--- Generating Final Submission CSV (Region-Specific ConvNeXt Regressors) ---")
submission_data = [] # List to hold (id, angle) tuples

# --- Define Test Dataset (Images only) ---
class SubmissionTestDatasetAngle(Dataset):
    def __init__(self, img_dir, transform, img_size=IMG_SIZE): # Use correct IMG_SIZE
        self.img_dir = img_dir; self.transform = transform; self.img_size = img_size
        try:
            self.image_files = glob.glob(os.path.join(self.img_dir, 'img_*.*'))
            if not self.image_files: print(f"Warning: No test images found in {self.img_dir}")
            self.image_files.sort(key=lambda f: int(os.path.splitext(os.path.basename(f))[0].split('_')[-1]))
            print(f"Found {len(self.image_files)} test images.")
        except Exception as e: print(f"Error finding/sorting test images: {e}"); self.image_files = []
    def __len__(self): return len(self.image_files)
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        try: image = Image.open(img_path).convert('RGB');
        except Exception: return torch.zeros((3, self.img_size, self.img_size)), img_path, False
        valid_load = True
        try:
            if self.transform: image = self.transform(image)
        except Exception: valid_load = False; image = torch.zeros((3, self.img_size, self.img_size))
        return image, img_path, valid_load # Image, Path, Valid Flag

# --- Create Datasets and Loaders for Submission ---
# Validation data (to get predictions in order)
df_val = pd.read_csv(VAL_CSV_PATH)
num_val_expected = len(df_val)
val_submission_dataset = AngleRegressionDataset(VAL_CSV_PATH, VAL_IMG_DIR, transform=transform_eval, is_train=False)
val_submission_loader = DataLoader(val_submission_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, collate_fn=angle_reg_collate_fn)
# Test data
test_submission_dataset = SubmissionTestDatasetAngle(TEST_IMG_DIR, transform_eval) # Use eval transform
test_submission_loader = DataLoader(test_submission_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# 1. Process Validation Set for Submission
print("Processing Validation Set for submission...")
val_preds_dict = {} # Store predictions by original index
with torch.no_grad():
    region_classifier_frozen.eval() # Ensure correct model is eval
    for batch_data in tqdm(val_submission_loader, desc="Val Submission Pred"):
        if batch_data is None: continue
        images, _, _, _, original_indices = batch_data # Need images and original indices
        images = images.to(DEVICE); indices_np = original_indices.cpu().numpy()
        region_logits = region_classifier_frozen(images); pred_region_ids = torch.argmax(region_logits, dim=1).cpu().numpy()
        for i in range(images.size(0)):
            pred_region_id = pred_region_ids[i]; original_idx = indices_np[i]
            regressor = trained_regressors.get(pred_region_id); pred_angle = 0.0
            if regressor is not None:
                try: regressor.eval(); outputs_cos_sin = regressor(images[i:i+1]); pred_angle_tensor = predictions_to_degrees(outputs_cos_sin); pred_angle = pred_angle_tensor.item();
                except Exception: pred_angle = 0.0
            if not np.isfinite(pred_angle): pred_angle = 0.0 # Ensure finite
            val_preds_dict[original_idx] = pred_angle
for i in range(num_val_expected): submission_data.append((i, val_preds_dict.get(i, 0.0)))
print(f"Collected {len(submission_data)} predictions for validation set.")

# 2. Process Test Set for Submission
print("\nProcessing Test Set for submission...")
num_test_images = len(test_submission_dataset); test_start_id = num_val_expected; test_predictions_temp = {}
if num_test_images > 0:
    with torch.no_grad():
        current_test_idx = 0; region_classifier_frozen.eval()
        for batch_data in tqdm(test_submission_loader, desc="Test Submission Pred"):
            if batch_data is None: current_test_idx += BATCH_SIZE ; continue
            images, img_paths, valid_loads = batch_data; num_in_batch = len(img_paths); indices_in_batch = list(range(current_test_idx, current_test_idx + num_in_batch))
            images_valid = images[valid_loads].to(DEVICE); indices_valid = [idx for idx, v in zip(indices_in_batch, valid_loads) if v]; indices_failed = [idx for idx, v in zip(indices_in_batch, valid_loads) if not v]
            for failed_idx in indices_failed: test_predictions_temp[failed_idx] = 0.0
            if images_valid.numel() > 0:
                region_logits = region_classifier_frozen(images_valid); pred_region_ids = torch.argmax(region_logits, dim=1).cpu().numpy()
                for i in range(images_valid.size(0)):
                    pred_region_id = pred_region_ids[i]; test_idx = indices_valid[i]
                    regressor = trained_regressors.get(pred_region_id); pred_angle = 0.0
                    if regressor is not None:
                        try: regressor.eval(); outputs_cos_sin = regressor(images_valid[i:i+1]); pred_angle_tensor = predictions_to_degrees(outputs_cos_sin); pred_angle = pred_angle_tensor.item();
                        except Exception: pred_angle = 0.0
                    if not np.isfinite(pred_angle): pred_angle = 0.0
                    test_predictions_temp[test_idx] = pred_angle
            current_test_idx += num_in_batch
    for test_idx in range(num_test_images): pred_angle = test_predictions_temp.get(test_idx, 0.0); submission_id = test_start_id + test_idx; submission_data.append((submission_id, pred_angle))
print(f"Collected predictions for {len(test_predictions_temp)} test samples (out of {num_test_images}).")

# 3. Finalize and Save CSV
print(f"\nTotal entries collected for submission: {len(submission_data)}")
submission_data.sort(key=lambda x: x[0]) # Sort by ID
temp_df = pd.DataFrame(submission_data, columns=['id','angle']); nan_count = temp_df['angle'].isna().sum()
if nan_count > 0:
    print(f"Found {nan_count} NaN angle predictions. Filling with 0.0"); fallback_angle = 0.0
    filled_submission_data = [];
    for sub_id, p_angle in submission_data: final_angle = fallback_angle if pd.isna(p_angle) else p_angle; filled_submission_data.append((sub_id, final_angle))
    submission_data = filled_submission_data
final_submission_df = pd.DataFrame(submission_data, columns=["id", "angle"]) # Use 'angle' column name
submission_filename = "submission.csv"
try:
    final_submission_df.to_csv(submission_filename, index=False, float_format='%.4f') # Format angle
    print(f"\nCombined submission file saved successfully to {submission_filename}")
    print(f"Total entries: {len(final_submission_df)}")
    if not final_submission_df.empty: print("Submission Head:\n", final_submission_df.head()); print("\nSubmission Tail:\n", final_submission_df.tail())
except Exception as e: print(f"\nERROR saving final submission file: {e}")

# --- Final Cleanup ---
gc.collect()
if DEVICE == 'cuda': torch.cuda.empty_cache()
print("\nScript finished.")

--- Configuration ---
Using device: cuda
Region Classifier Arch: convnextv2_tiny.fcmae_ft_in1k
Regressor Base Arch: convnextv2_tiny.fcmae_ft_in1k
Image Size: 224x224
Epochs per Region Model: 30
Regressor LR: 0.0001
Min Samples per Region: 20
Train CSV Path: /kaggle/input/iiith-campus/labels_train.csv
Val CSV Path: /kaggle/input/iiith-campus/labels_val.csv
Train Image Dir: /kaggle/input/iiith-campus/images_train/images_train
Val Image Dir: /kaggle/input/iiith-campus/images_val/images_val
Test Image Dir: /kaggle/input/images-test/images_test
---------------------

Loading Frozen Region Classifier: convnextv2_tiny.fcmae_ft_in1k


/tmp/ipykernel_31/2247342287.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(REGION_CLASSIFIER_PATH, map_location='cpu'); state_dict = checkpoin

Region classifier loaded and frozen successfully.
Defining Transformations...
 Transforms created (Input Size: 224x224). Includes Training Augmentation.

Loading full training dataset...
Filtered training data: 6542 -> 6540 rows.
Loading full validation dataset...
Full training dataset size: 6540
Full validation dataset size: 369

--- Training 15 Region-Specific ConvNeXt Regressors ---

-- Training Regressor for Region 1/15 --
 Training with 362 samples.
 Validating with 21 samples for this region.


   ---> New best Val MAAE for region 1: 67.4868 at epoch 1
  Reg 1 - Ep 1/30 - Train Loss(MSE): 0.4702 | Train MAAE: 69.7466 | Val MAAE: 67.4868


   ---> New best Val MAAE for region 1: 59.1311 at epoch 2
  Reg 1 - Ep 2/30 - Train Loss(MSE): 0.3044 | Train MAAE: 41.5897 | Val MAAE: 59.1311


...


Final Overall Validation MAAE (All Samples): 25.3449

--- Generating Final Submission CSV (Region-Specific ConvNeXt Regressors) ---
Found 369 test images.
Processing Validation Set for submission...


Val Submission Pred: 100%|██████████| 12/12 [00:06<00:00,  1.90it/s]


Collected 369 predictions for validation set.

Processing Test Set for submission...


Test Submission Pred: 100%|██████████| 12/12 [00:06<00:00,  1.84it/s]


Collected predictions for 369 test samples (out of 369).

Total entries collected for submission: 738

Combined submission file saved successfully to submission.csv
Total entries: 738
Submission Head:
    id       angle
0   0  324.957886
1   1   70.141296
2   2  173.504578
3   3  306.471344
4   4  319.156677

Submission Tail:
       id       angle
733  733  202.921677
734  734   71.362671
735  735  218.981369
736  736  227.592712
737  737  225.291428

Script finished.


In [6]:
import os
import torch

# --- Configuration for Saving ---
# Assume WORK_DIR and REGRESSOR_ARCH are defined from the previous cell
# Example fallback definitions if running this cell independently:
if 'WORK_DIR' not in locals():
    WORK_DIR = "/kaggle/working/"
    print(f"Warning: WORK_DIR not defined, using default: {WORK_DIR}")
if 'REGRESSOR_ARCH' not in locals():
    REGRESSOR_ARCH = 'convnextv2_tiny.fcmae_ft_in1k'
    print(f"Warning: REGRESSOR_ARCH not defined, using default: {REGRESSOR_ARCH}")

# Directory to save the 15 model files
REGION_MODEL_SAVE_DIR = os.path.join(WORK_DIR, "region_specific_angle_models")
os.makedirs(REGION_MODEL_SAVE_DIR, exist_ok=True)
print(f"Saving region-specific models to: {REGION_MODEL_SAVE_DIR}")

# Assume 'trained_regressors' dictionary exists and contains the trained models
if 'trained_regressors' not in locals() or not isinstance(trained_regressors, dict):
    print("Error: 'trained_regressors' dictionary not found or is not a dictionary.")
    print("       Please ensure the training cell was executed successfully.")
else:
    # --- Loop through the trained models and save each one ---
    saved_count = 0
    skipped_count = 0
    for region_id, model in trained_regressors.items():
        if model is not None:
            # Define a descriptive filename for each model
            model_filename = f"region_{region_id+1:02d}_{REGRESSOR_ARCH.split('.')[0]}_angle_regressor.pth"
            save_path = os.path.join(REGION_MODEL_SAVE_DIR, model_filename)
            try:
                # Ensure model is on CPU before saving state_dict for better compatibility
                model.cpu()
                torch.save(model.state_dict(), save_path)
                print(f" Saved model for Region {region_id+1} to {save_path}")
                saved_count += 1
                # Optional: Move model back to original device if needed later
                # model.to(DEVICE)
            except Exception as e:
                print(f" Error saving model for Region {region_id+1}: {e}")
        else:
            print(f" Skipping Region {region_id+1}: No model was trained (likely due to insufficient data).")
            skipped_count += 1

    print(f"\nFinished saving models. Saved: {saved_count}, Skipped: {skipped_count}")

# Optional: Verify saved files
print("\nVerifying saved files:")
saved_files = os.listdir(REGION_MODEL_SAVE_DIR)
print(f" Found {len(saved_files)} files in {REGION_MODEL_SAVE_DIR}:")
for f in saved_files[:15]: # Print first few files
    print(f"  - {f}")

Saving region-specific models to: /kaggle/working/region_specific_angle_models
 Saved model for Region 1 to /kaggle/working/region_specific_angle_models/region_01_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 2 to /kaggle/working/region_specific_angle_models/region_02_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 3 to /kaggle/working/region_specific_angle_models/region_03_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 4 to /kaggle/working/region_specific_angle_models/region_04_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 5 to /kaggle/working/region_specific_angle_models/region_05_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 6 to /kaggle/working/region_specific_angle_models/region_06_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 7 to /kaggle/working/region_specific_angle_models/region_07_convnextv2_tiny_angle_regressor.pth
 Saved model for Region 8 to /kaggle/working/region_specific_angle_models/region_